In [3]:
import scipy.io as io
# read data/zuco/task1/Matlab_files/resultsZAB_SR.mat
sentence_data = io.loadmat('data/zuco/task1/Matlab_files/resultsZAB_SR.mat', squeeze_me=True, struct_as_record=False,)['sentenceData']

In [6]:
import numpy as np
import pandas as pd
sn_data = sentence_data[0]
dur = np.array(sn_data.allFixations.duration)
x = np.array(sn_data.allFixations.x)
y = np.array(sn_data.allFixations.y)


In [12]:
sn_data.content

'Presents a good case while failing to provide a reason for us to care beyond the very basic dictums of human decency.'

In [ ]:
word_boundary = sn_data.wordbounds
df = pd.DataFrame([], columns=['sn', 'wn', 'fl', 'dur'])
for fix_idx in range(x.shape[0]):
    word_idx = np.where(
        (word_boundary[:, 0] <= x[fix_idx]) &
        (word_boundary[:, 2] >= x[fix_idx]) &
        (word_boundary[:, 1] <= y[fix_idx]) &
        (word_boundary[:, 3] >= y[fix_idx]),
    )
    assert len(word_idx) == 1, 'more than 1 word is matched!'
    # skip fixations outside all boundary boxes
    if word_idx[0].size == 0:
        continue
    fl = ((x[fix_idx] - word_boundary[word_idx, 0]) /
            (word_boundary[word_idx, 2] - word_boundary[word_idx, 0]))[0][0]
    df_tmp = pd.DataFrame(
        [[
            1,  word_idx[0][0] + 1,
            fl, dur[fix_idx],
        ]], columns=['sn', 'wn', 'fl', 'dur'],
    )
    df = pd.concat([df, df_tmp])


In [11]:
words = pd.read_csv("data/zuco/task1/Matlab_files/scanpath.csv", sep='\t')
# check is there is nan in words
words.isnull().values.any()

# check if there is nan or '' in column 'WORD'
words[words['CURRENT_FIX_INTEREST_AREA_LABEL'].isnull().values]
words[words['CURRENT_FIX_INTEREST_AREA_LABEL'] == '']

,id,sn,nw,wn,fl,dur,CURRENT_FIX_INTEREST_AREA_LABEL
